## Procesamiento de Datos

![logo](img/logo.jpeg)


# Sumario
- Trabajo con strings
- Combinando datasets
- Limpieza de datos
 - map, filter, reduce
 - filling missing values
 - valores duplicados
 - categorizacion de datos

In [1]:
import numpy as np
import pandas as pd

### Descargar y descomprimir   

In [2]:
import zipfile as zp # para descomprimir archivos zip
import urllib.request # para descargar de URL
import os

# descargar MovieLens dataset
url = 'http://files.grouplens.org/datasets/movielens/ml-1m.zip'  
local_zip = os.path.join("res", "ml-1m.zip")
urllib.request.urlretrieve(url, local_zip)
# descomprimiendo archivo zip
with zp.ZipFile(local_zip, 'r') as zipp: 
    print('Descomprimiendo ficheros...') 
    zipp.extractall(os.path.join("res")) # destino
    print('Hecho!') 

### Combinar varios datasets 
- En base a un elemento en común (índice)
- MovieLens 'UserId'

In [3]:
root_path = os.path.join("res", "ml-1m" )

ratings_dataset = pd.read_csv(os.path.join(root_path, "ratings.dat"), sep='::',
                                index_col=0, engine='python',
                                names=['UserID','MovieID','Rating','Timestamp'])

users_dataset = pd.read_csv(os.path.join(root_path, "users.dat"),sep='::',
                              index_col=0, engine='python',
                              names=['UserID','Gender','Age','Occupation','Zip-code'])

In [4]:
users_dataset.sample(5)

In [5]:
ratings_dataset.sample(5)

### Uniendo datasets con 'join' y 'merge'
- merge() == join()
 - 'join' utiliza por defecto los índices para unir
- Utilizando el parámetro 'on'
 - Si las columnas difieren, 'left_on' y 'right_on'
 
 https://i.stack.imgur.com/hMKKt.jpg

Para combinar datasets podemos usar ```merge()```, estableciendo que columna se usará como 'enlace' con el parámetro **on** y especificando el tipo de 'join' con el parámetro **how**.   

Ejemplo:

In [ ]:
# Combinando users y ratings, ¿Cómo?
# Con merge(), especificamos el dataset con el que queremos combinar, y la columna que se usará como pivote, en este caso 'UserID'
# También especificamos el tipo de combinación, en este caso 'inner', que solo incluirá los registros que tengan un valor en ambas tablas
combined_dataset = users_dataset.merge(ratings_dataset, on='UserID', how='inner') 
display(combined_dataset.sample(5))
len(combined_dataset)

In [ ]:
# Visualizando el dataset de películas (movies)
movies_dataset = pd.read_csv(os.path.join(root_path, "movies.dat"),sep='::',encoding='latin-1', engine='python',names=['MovieID','Title','Genre'])
movies_dataset.sample(5)

In [ ]:
# Combinando movies y el resto
# Tomamos el combined_dataset y lo unimos con movies_dataset, usando 'MovieID' como pivote
# También este caso, usamos 'inner' para que solo se incluyan los registros que tengan un valor en ambas tablas
all_dataset = combined_dataset.merge(movies_dataset,on='MovieID', how='inner')
all_dataset.sample(5)

## Concatenación con PANDAS (```concat()```)
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

El método ```concat()``` de Pandas, es un método que nos permite concatenar objetos de Pandas (Series o DataFrames) a lo largo de un eje en particular.   
 
A continuación veremos un pequeño ejemplo sobre como funciona este método y cómo podemos utilizarlo para concatenar información:

In [47]:
import pandas as pd

cliente_uno = pd.DataFrame({
    'Identificador': [1, 2, 3, 4, 5],
    'Nombre': ['James', 'Emma', 'Liam', 'Olivia', 'William'],
    'Edad': [25, 30, 22, 24, 32],
    'Email': ['james@email.com', 'emma@email.com', 'liam@email.com', 'olibia@email.com', 'william@email.com'],
    'Telefono': ['1234567890', '0987654321', '1230984567', '1234567893', '1237897654'],
})

clientes_dos = pd.DataFrame({
    'Identificador': [6, 7, 8, 9, 10],
    'Nombre': ['Jane', 'Henry', 'Alexander', 'Mia', 'Ava'],
    'Edad': [28, 35, 26, 27, 40],
    'Email': ['janet@gmail.com', 'henry@email.com', 'alexander@email.com', 'mia@gmail.com', 'ava@email.com'],
})

clientes_totales = pd.concat([cliente_uno, clientes_dos], ignore_index=True)
print(clientes_totales)

Este método permite concatenar dos objetos de Pandas a lo largo de cualquier eje, verticalmente u horizontalmente. En el anterior ejemplo se concatenan los dos Dataframes verticalmente y además se puede ver que al segundo Dataframe, en la columna Telefono, le coloca el valor de NaN. Esto sucede porque el primer DataFrame tiene valores en esta columna pero el segundo no.

### 1. Concatenando por FILAS (Verticalmente)

En el siguiente ejemplo, concatenamos los dos Dataframes de usuarios utilizando el método ```concat()``` y le pasamos los dos Dataframes en una lista.    

Esto creará un nuevo Dataframe con la información de todos los usuarios.

In [ ]:
import pandas as pd

users_us = pd.DataFrame({
    'User_id': [1, 2, 3, 4, 5],
    'Username': ['Alice', 'Bob', 'Charlie', 'Jane', 'Thomas'],
    'Age': [25, 30, 22, 24, 34]
})

users_mx = pd.DataFrame({
    'User_id': [6, 7, 8],
    'Username': ['Axel', 'Camilo', 'Ariana'],
    'Age': [28, 35, 26]
})

all_users = pd.concat([users_us, users_mx], ignore_index=True) # el parametro ignore_index=True, es para que los indices se reasignen de forma secuencial.  
print(all_users)

### 2. Concatenando por COLUMNAS (Horizontalmente)

En el siguiente ejemplo tenemos tres diferentes Dataframes:   

- el primero contiene varios países de América, 
- el segundo países de Europa y 
- el tercero países de Asia.    

Para concatenar estos tres Dataframes horizontalmente, simplemente necesitamos usar el método ```concat()``` con dos parámetros:   

- el primero son los tres Dataframes dentro de una lista y 
- el segundo parámetro es ```axis=1``` que le indica al método que tiene que hacer la concatenación horizontalmente.

In [ ]:
import pandas as pd

countries_america = pd.DataFrame({
    'Country_code': ['USA', 'CAN', 'MX', "COL"],
    'Population': [330, 29, 38, 67]
})

countries_europe = pd.DataFrame({
    'Country_code': ['GER', 'FRA', 'ITA'],
    'Population': [83, 67, 60]
})

countries_asia = pd.DataFrame({
    'Country_code': ['CN', 'IN', 'JP'],
    'Population': [57, 77, 63]
})
# concatenamos horizontalmente los dataframes
all_countries = pd.concat([countries_america, countries_europe, countries_asia], axis=1)
print(all_countries)

### 3. Concatenar Dataframes con claves de nivel superior (MultiIndex)   

En el ejemplo siguiente tenemos un conjunto de Dataframes similar al ejemplo anterior, pero en este caso queremos concatenarlos verticalmente y colocarle una clave a cada uno de ellos.    

Para esto, simplemente debemos pasarle al método ```concat()``` la lista de Dataframes y además el parámetro ```keys``` con una lista que contine las claves para cada uno de los Dataframes.

In [50]:
import pandas as pd

countries_america = pd.DataFrame({
    'Country_code': ['USA', 'CAN', 'MX', "COL"],
    'Population': [330, 29, 38, 67]
})

countries_europe = pd.DataFrame({
    'Country_code': ['GER', 'FRA', 'ITA'],
    'Population': [83, 67, 60]
})

countries_asia = pd.DataFrame({
    'Country_code': ['CN', 'IN', 'JP'],
    'Population': [57, 77, 63]
})

all_countries = pd.concat([countries_america, countries_europe, countries_asia], keys=['America', 'Europa', 'Asia'])
print(all_countries)

### 4. Concatenar Series a lo largo de las filas.   

El método ```concat()``` también nos permite concatenar Series.    

En el siguiente ejemplo disponemos de dos Series con el id de varios usuarios. Para concatenarlos simplemente necesitamos utilizar el método ```concat()``` y pasarle la lista de Series que queremos concatenar. También se puede hacer uso de los demás parámetros para modificar la concatenación según sea necesario.

In [51]:
import pandas as pd

user_ids_uno = pd.Series([1, 2, 3, 4, 5], name='User_id')
user_ids_two = pd.Series([6, 7, 8, 9, 10], name='User_id')

all_users = pd.concat([user_ids_uno, user_ids_two], ignore_index=True)

print(all_users)

## Método ```pivot()``` de PANDAS   

El método ```pivot()``` de la librería de Pandas nos permite reorganizar y transformar los datos de un DataFrame creando una nueva tabla con un formato diferente.     

El método ```pivot()``` es un método de la librería de Pandas que nos permite transformar los datos de un DataFrame al reorganizar sus datos en función de las columnas existentes.    
Permite reconfigurar los datos de manera que los valores en una columna se conviertan en nuevas columnas y se crucen con los valores de otra columna.    
Esto es especialmente útil para crear tablas dinámicas y resúmenes de datos.

El método ```pivot()``` se utiliza principalmente en situaciones en las que se desea cambiar la estructura de los datos para un análisis más conveniente.    
Permite que los datos sean más legibles y accesibles al proporcionar una vista diferente de los mismos.    
Este método retorna un nuevo DataFrame con los datos pivotados y no modifica el DataFrame original.

In [59]:
import pandas as pd

products_df = pd.DataFrame({
    'Fecha': ['2023-01-01', '2023-01-01', '2023-01-02', '2023-01-02'],
    'Producto': ['Samsung', 'Apple', 'Samsung', 'Apple'],
    'Venta': [100, 150, 200, 120]
})

print("Información original:")
print(products_df)

pivot_df = products_df.pivot(index='Fecha', columns='Producto', values='Venta')
print("\nInformación pivoteada:")
print(pivot_df)

### Parámetros del método ```pivot()```  

El método ```pivot()``` recibe tres parámetros, siguiendo la estructura que se muestra a continuación:

```data_frame.pivot(index, columns, values)```  

- **index**: Este parámetro recibe como valor una columna o lista de columnas que se usan como índices en el nuevo DataFrame. Puede ser una cadena o una lista de cadenas. Si se omite, se usa el índice de DataFrame original.
- **columns**: (required) Este parámetro recibe como valor la columna o lista de columnas que se usan como los nombre para las columnas en el nuevo DataFrame.
- **values**: Este parámetro recibe como valor la columna o lista de columnas que se usan como los valores para el nuevo DataFrame. Si no se especifica, se utilizarán todas las columnas restantes y el resultado tendrá columnas indexadas jerárquicamente.

#### 1. Utilizar una sola columna para crear el dataframe.   

En el siguiente ejemplo, se le pasa una sola columna como valor al parámetro ```index``` y al parámetro ```columns```:

In [60]:
import pandas as pd

df = pd.DataFrame({
    "producto": ["Apple", "Apple", "Apple", "Samsung", "Samsung", "Samsung", "Linux", "Linux", "Linux"],
    "pais": ["Colombia", "Perú", "Ecuador", "Colombia", "Perú", "Ecuador", "Colombia", "Perú", "Ecuador"],
    "año": [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024],
    "ventas": [1000, 800, 600, 1200, 900, 700, 1100, 850, 650]
})
print(df)


In [61]:
df_pivot = df.pivot(index="año", columns="producto", values="ventas").fillna("N/A")

print(df_pivot)

En el anterior ejemplo se hace uso del método ```pivot()``` para transformar un DataFrame de productos y se utilizan los valores de la columna año como índice, los valores de la columna producto para representar las columnas y los valor de la columna ventas para llenar los valores en el nuevo DataFrame.    
Además se usa el método ```fillna()``` para reemplazar todos los valores NaN con el texto **N/A(No aplica)**.    

En definitiva, se ha transformado el DataFrame para ver cuántas ventas ha tenido cada producto en cada año.

#### 2. Utilizar una lista de columnas para crear el dataframe   

El método ```drop()``` también puede recibir una lista de columnas como valores para los parámetros.    

El siguiente es un ejemplo sobre cómo puede ser útil en algunas ocasiones:

In [62]:
import pandas as pd

df = pd.DataFrame({
    "producto": ["Apple", "Apple", "Apple", "Samsung", "Samsung", "Samsung", "Linux", "Linux", "Linux"],
    "pais": ["Colombia", "Perú", "Ecuador", "Colombia", "Perú", "Ecuador", "Colombia", "Perú", "Ecuador"],
    "año": [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024],
    "ventas": [1000, 800, 600, 1200, 900, 700, 1100, 850, 650]
})

df_pivot = df.pivot(index="año", columns=["producto", "pais"], values="ventas").fillna("N/A")

print(df_pivot)

Como se puede observar, se hace uso del método ```pivot()``` y se le pasa una lista como valor al parámetro ```columns```.    
Esta lista contiene dos columnas (producto y pais) lo que significa que el método ```pivot()``` utilizará la columna producto y creará una columna con cada uno de sus valores.    
Luego creará una subcolumna con los valores de la columna pais y agrega esta subcolumna a cada una de las columnas de Producto.    
Por último hacemos uso del método ```fillna()``` para reemplazar todos los valores NaN por el texto N/A(No aplica).    


Este ejemplo puede ser un poco confuso, pero utilizar una lista de columnas como valores para los parámetros puede ser muy útil en determinadas ocasiones.

## Método ```pivot_table()```   

La principal función de ```pivot_table()``` son las agrupaciones de datos, a las que se les suelen aplicar funciones matemáticas como sumatorios, promedios, etc.    
Si no indicamos en el parámetro ```aggfunc``` que opereción queremos hacer, por defecto, nos calculará la ***media*** de todas aquellas columnas que sean de tipo numérico.

Estructura del método:   

```python
pivot_table(<lista de valores>, index=<agregador primario>, columns=<agregador secundario>)
```
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html

In [58]:
# all_dataset.pivot_table('Rating', index='Gender', columns='Age')
# all_dataset.pivot_table('Rating', index='Gender', columns='Age', aggfunc='count')
all_dataset.pivot_table('Rating', index='Gender', columns='Age', aggfunc=['count', 'mean']) # cuenta por sexo y edad

## Agrupaciones
- agg -> funciones estadísticas de agregación
- Series.unique() -> valores únicos
- pd.value_counts -> ocurrencias

## Manipulación de strings
```python
split(): separar en bloques en función de un carácter
replace(): reemplazar un carácter por otro
index(): encontrar la posición de un carácter
```

In [10]:
# Ejemplo con MovieLens: Genre
## 1: obtener todos los géneros por separado
## 2: crear un dataset de géneros
## 3: por película, marcar género por separado
## 4: unir con dataset original
movies_dataset.head(3)

#### 1. Obtener todos los géneros de forma separada

In [63]:
# Realizamos un split de la columna 'Genre' para obtener los géneros por separado, usando una función lambda
# El carácter que usamos para separar es '|'.
all_genres = movies_dataset['Genre'].apply(lambda x : x.split('|'))
print(all_genres)

# print([genre for x in all_genres for genre in x])

# Con pd.unique() obtenemos los valores únicos de una lista y, en este caso, 
# los géneros únicos, a través del bucle que generamos dentro de unique()
genres = pd.unique([genre for x in all_genres # bucle para cada género (genre) en la lista de géneros (all_genres)
                    for genre in x]) # bucle para cada género (genre) dentro de la lista de géneros (x)
display(genres)

In [12]:
# crear tabla con columnas por género
zeros = np.zeros( (len(movies_dataset), len(genres)) )
genres_frame = pd.DataFrame(zeros, columns=genres)
genres_frame.head(3)

#### 2. Crear dataset de genéros

In [64]:
columns_genres = genres_frame.columns # lista de generos (columnas)
print(columns_genres)


#### 3. Separar los distintos géneros para cada película.   

A continuación, el código se encarga de convertir una lista de géneros en formato de cadena con "|" en una matriz one-hot encoding, donde cada película tiene un 1 en las columnas que corresponden a sus géneros y 0 en las demás.

In [ ]:


# enumerate(movies_dataset['Genre']) permite iterar sobre cada fila de la columna 'Genre', 
# proporcionando tanto el índice i (posición de la película en el dataset) como el contenido genre (cadena de texto con los géneros de esa película).
for i, genre in enumerate(movies_dataset['Genre']): # Iteración sobre cada fila de la columna 'Genre'
    # Obtener los índices de los géneros en la matriz OHE (one-hot-encoding) de géneros
    inds = columns_genres.get_indexer(genre.split('|')) # get_indexer() retorna los indices de los generos en la lista de generos 'genre.split('|')', que convierte la cadena "Action|Comedy" en la lista ['Action', 'Comedy'].
    # actualiza la fila i (película en cuestión), estableciendo 1 en las posiciones inds (las columnas correspondientes a los géneros de esa película).
    # Es decir, convierte la información de la columna 'Genre' en un formato de codificación one-hot.
    genres_frame.iloc[i,inds] = 1 # localiza las columnas del genero correspondiente, marca con 1

In [14]:
genres_frame.head(5)

#### 4. Unir con dataset original.   

En el siguiente paso, se une el nuevo dataset con el dataset original de películas mediante ```join()```para obtener el dataset completo con los géneros segregados

In [15]:
# unir con dataset original
movies_split_genre = movies_dataset.join(genres_frame)

In [16]:
display(movies_split_genre.head(5))

### Extracción del año de la película usando ```replace()``` e ```index()```

In [17]:
movies_dataset.head(2)

In [ ]:
# extraer el año de la columna Title
def split_year(title):
    index = title.index('(')  # establecemos el 'punto de partida' para sacar el año, usando el método index() para encontrar el primer paréntesis
    return title[index:].replace('(','').replace(')','') #sustituimos los paréntesis por 'nada' con replace()
    
# crear nueva columna Year
movies_dataset['Year'] = movies_dataset['Title'].apply(split_year) # aplicamos la función split_year() (acabada de crear) sobre la columna 'Title' y guardamos el resultado en la nueva columna 'Year'
display(movies_dataset.sample(2))

In [ ]:
# eliminar el año de la columna Title
def remove_year(title):
    index = title.index('(') # establecemos el 'punto de partida' para sacar el título, usando el método index() para encontrar el primer paréntesis
    return title[:index-1].strip() # eliminamos el año y los espacios en blanco al principio y al final con strip()

movies_dataset['Title'] = movies_dataset['Title'].apply(remove_year) # aplicamos la función remove_year() (acabada de crear) sobre la columna 'Title' para eliminar el año de los títulos
movies_dataset.head(2)

## Expresiones regulares
https://docs.python.org/3/library/re.html

- import re

### ¿Cómo localizar que 'Zip-code' tiene un formato erróneo?

In [ ]:
users_dataset.sample(5)

# Formato válido:
# ^\d{5}$
# Donde:
# ^ = start of the string
# \d = decimal string
# {5} = 5 repeticiones de decimales
# $ = end of string

In [66]:
users_dataset[users_dataset['Zip-code'].str.match('^\d{5}$') == True] # localizamos los códigos postales que cumplen con el formato correcto

In [67]:

users_dataset[users_dataset['Zip-code'].str.match('^\d{5}$') == False] # muestra los registros que no cumplen con el formato de 5 dígitos

### ¿Cómo extraer el año con expresiones regulares usando el formato adecuado?

In [ ]:
movies_dataset = pd.read_csv(os.path.join(root_path, "movies.dat"),sep='::', engine='python',encoding='latin-1',names=['MovieID','Title','Genre'])
display(movies_dataset.head(2))

# (\d{4}) -> expresión regular para encontrar el año en el título
# Donde:
# (= busca apertura parentesis
# \d = decimal string
# {4} = 4 repeticiones de decimales
# ) = cierre de parentesis

In [69]:
movies_dataset['Title'].str.extract('(\d{4})') # extraemos los años de la columna 'Title' 

### Usando la librería ```re```

La expresión regular del código siguiente se debe interpretar cómo:   
- r"^ = inicio de la cadena, 
- ( = agrupación, 
- (?! = negación de la siguiente expresión, 
- English = la palabra 'English', 
- . = cualquier carácter, 
- )* = cualquier cantidad de veces, 
- $ = fin de la cadena

In [ ]:
import re  # importamos el módulo 're' para trabajar con expresiones regulares

test_str = ("English 101 Class A\n"
	"English 201 Class B\n"
	"Spanish 101 Class D\n"
	"Italian 201 Class E\n"
	"French 101 Class F\n")

def searchAllButEnglish(text):
  regex = r"^((?!English).)*$"                      # expresión regular para encontrar todas las clases excepto las de inglés. 
  matches = re.finditer(regex, text, re.MULTILINE)  # buscamos todas las coincidencias en el texto
  for match in matches:                             # iteramos sobre las coincidencias
    print(match)                                    # imprimimos la coincidencia
  return match
print(searchAllButEnglish(test_str))

#### Ejemplo sencillo del uso de ```re```para expresiones regulares

In [86]:
texto= "Este es mi texto de prueba en el que voy a querer cambiar unas palabras por Perro. Simplemente buscado coincidencias para perro";

regex= r"[Pp]erro" # expresión regular para encontrar la palabra 'perro' o 'Perro'

matches = re.findall(regex,texto) # buscamos todas las coincidencias en el texto
print(matches)

## Operaciones con colecciones

- ```reduce```: aplicar una operación y retornar un valor
- ```filter```: retorna una secuencia con elementos que cumplen una condición
- ```map```: aplicar  una operación y retornar una secuencia


### Reduce
- Aplicar una operación matemática a cada uno de los elementos de una colección
- Diferente de 'apply()' porque retorna un valor numérico
- Ejemplo: Detección de géneros en años específicos

https://docs.python.org/3/library/functools.html

```reduce``` es muy útil cuando queremos realizar ciertas operaciones sobre una lista y devolver su resultado.    
Por ejemplo, si queremos calcular la suma de todos los elementos de una lista, y devolver un único valor, podríamos hacerlo de la siguiente forma usando ```reduce```:

In [ ]:
from functools import reduce # necesario para reduce

lista = [1, 3, 5, 7, 9]
print(reduce(lambda x,y: x + y, lista)) # suma de todos los elementos de la lista

Preparamos un nuevo ejemplo en el que, primero localizamos y creamos un dataset con las películas del año 1975

In [101]:
movies_1975 = movies_split_genre[ movies_split_genre['Title'].str.contains('1975') ]
movies_1975.head(3)

Ahora vamos a averiguar si existe alguna película que pertenezca al género "Drama" dentro de ese dataset

In [102]:
any_drama = reduce(lambda x,y : bool(x) | bool(y),movies_1975['Drama']) # hay algún drama en 1975
print(any_drama)



En el siguiente bloque de código se comprueba si todas las películas del año 1975 son del género "Comedy"

In [103]:
all_comedy = reduce(lambda x,y : bool(x) & bool(y),movies_1975['Comedy']) # son todas las películas de 1975 comedias?
print(all_comedy)

La siguiente línea nos permite saber ***si existe algún valor*** que cumpla la condición de ser del género seleccionado

In [104]:
print(movies_1975['Drama'].any()) # Comprueba si hay algún valor que puede cumplir  

En el siguiente caso, la comprobación es ***si todos los valores*** del dataset cumplen la condición de ser del género "Comedy"

In [105]:
print(movies_1975['Comedy'].all()) # Comprueba si todos los valores son True

Ahora comprobaremos el número de entradas que existe en el dataset que cumplan la condición de tener el género "Comedy"

In [106]:
# Observar el tipo de dato antes para ver si es posible aplicar las funciones
print(movies_1975.dtypes)

In [108]:
print(movies_1975['Comedy']) # esto muestra por pantalla la columna 'Comedy'

In [ ]:
print(movies_1975['Comedy'].value_counts()) # esto muestra por pantalla la cantidad de entradas únicas en la columna 'Comedy' (con valor 1.0) = 6 entradas.

### Filter   

La función ```filter``` crea una lista de elementos si usados en la llamada a una función devuelven ```True``` Es decir, filtra los elementos de una lista usando un determinado criterio.

- retorna una secuencia con elementos que cumplen una condición.    

La función ```filter``` es similar a un bucle ( se podría conseguir lo mismo con un bucle y un ```if```) pero su uso es más rápido.


In [112]:
lista = range(-5, 5)
menor_cero = list(filter(lambda x: x < 0, lista))
print(menor_cero)

Para nuestro dataset de películas: obtener las películas de 1975 que contienen 'The' en el título

In [113]:
filtro = filter(lambda x : 'The' in x, movies_1975['Title']) # filtramos las películas de 1975 que contienen 'The' en el título.
list(filtro)

### Map   

El uso de ```map``` aplica una determinada función/operación a todos los elementos de una entrada o lista, retornando una secuencia. Esta es su forma:   

```map(funcion_a_aplicar, lista_de_entradas)```

Un sencillo ejemplo:

In [114]:
lista = [1, 2, 3, 4, 5]
al_cuadrado = list(map(lambda x: x**2, lista))
print(al_cuadrado)

De nuevo, sobre el dataset películas, se desea cambiar el valor integral de la columna 'Comedy' por bool:

In [ ]:
mapa = map(lambda x : bool(x), movies_1975['Comedy'])   # mapeamos la columna 'Comedy' de movies_1975 a booleanos, generando una lista de valores booleanos.
mapa1 =map(lambda x : bool(x), movies_1975['Comedy'])   # duplicamos la línea anterior para mostrar el resultado, ya que al aplicar map() no se ejecuta la función hasta que se solicita.

print(list(mapa1))

In [127]:
movies_1975.loc[:,'Comedy'] = list(mapa)                # actualizamos la columna 'Comedy' con los valores booleanos
movies_1975.head(4) 

Otra forma de usar ```map``` es combinando una lista de funciones en lugar de una sola. Veamos un ejemplo:

In [ ]:
def multiplicar(x):
    return (x*x)
def sumar(x):
    return (x+x)

funcs = [multiplicar, sumar]
for i in range(5):
    valor = list(map(lambda x: x(i), funcs)) # nos devuelve la multiplicación y la suma de cada valor de i (0, 1, 2, 3, 4)
    print(valor)

## Transformación de variables (calidad de datos)
- Tratamiento de valores no definidos
- Tratamiento de valores duplicados
- Discretización (valores categóricos)

### Tratamiento de valores no definidos (NaN, null,...)

In [ ]:
# Preparación del dataframe de ejemplo
matrix = pd.DataFrame(np.random.randint(10,size=(5,10)))    # creamos un DataFrame de 5x10 con valores aleatorios entre 0 y 9
matrix[matrix < 2] = np.nan                                 # reemplazamos los valores menores a 2 por NaN
matrix

#### Mostrar cantidad de valores nulos por columnas

In [ ]:
# nulos por columna
matrix.isnull().sum() 


In [ ]:
# Si usamos la función isna() en lugar de isnull(), obtendremos el mismo resultado
matrix.isna().sum()

#### Cantidad total de valores nulos en el dataframe matrix

In [131]:
# Cantidad valores nulos
matrix.isnull().sum().sum() 

#### Recuento de valores no nulos por fila

In [132]:
# numero de no nulos por fila
matrix.count(axis=1)

#### Recuento de valores nulos por fila

In [133]:
# Número de nulos por fila
matrix.shape[1] - matrix.count(axis=1) # tomamos el número de columnas y restamos el número de valores no nulos por fila

#### Mostrar filas que tienen alguna columna (la indicada) con valores determinados.

In [ ]:
# Representación de las filas en las que una determinada columna tiene nulos
matrix[matrix[6].isnull()] # muestra las filas en las que la columna 6 tiene valores nulos

#### Mostrar filas en las que una determinada columna contiene un conjunto concreto de valores.

In [ ]:
valores = [8, 4] # valores a buscar
matrix[matrix[6].isin(valores)] # muestra las filas en las que la columna 6 tiene valores 8 o 4

#### Eliminar valores nulos

In [36]:
## Tratamiento de valores nulos
# eliminar
matrix.dropna()

Usando el parámetro ```thresh```(Umbral)

In [ ]:
# eliminar si no hay un número de valores no NaN
matrix.dropna(thresh=7) # elimina las filas que tienen menos de 7 valores no NaN

#### Sustituir/Rellenar nulos por un determinado valor

In [38]:
# sustituir por un valor fijo
matrix.fillna(-1)

#### Sustitución/Relleno dinámico.   

sustituir por valor dinámico...
- bfill -> backward fill (relleno hacia atrás)
- ffill -> forward fill (relleno hacia adelante)

In [140]:

print(matrix)
matrix.fillna(method='bfill') # bfill y ffill

#### Sustitución por interpolación

In [142]:
# sustituir por valor dinámico (interpolación)
print(matrix)
matrix.interpolate() # interpolación lineal. Establece los valores NaN en función de los valores adyacentes, asignando un valor intermedio, en este caso, la media entre los valores adyacentes.
print(matrix.interpolate())

#### Tratar valores duplicados

In [ ]:
serie = pd.Series(['a','b','c','a','c','a','g'])
serie.duplicated() # muestra los valores duplicados

In [144]:
df = all_dataset
df
# eliminar
# Eliminación de los duplicados en una columna definida
df2 = df.drop_duplicates(subset="Gender", keep='last', inplace=False)
display(df2)

#### Discretización (valores categóricos)
- Tras Series y DataFrame, objeto para categorías: Categorical
```python
categorias = pd.cut(<valores>, <bins>) 
```

In [145]:
# especificar los bloques
bins = [0,18,35,65,99]
edades = [16,25,18,71,44,100,12]
categorias = pd.cut(edades,bins) # cut divide los valores en los bloques especificados
print(categorias)

In [146]:
categorias.value_counts() # muestra la cantidad de valores en cada bloque

In [ ]:
# especificar el número de bloques
bins = 5                         # número de bloques. En este caso, 5 bloques que no se especifican, sino que se generan automáticamente, de forma que tengan una distancia similar entre ellos.
edades = [0,6,8,16,25,18,71,44,100]
categorias = pd.cut(edades,bins) # rangos idénticos (similar distancia de rangos)
print(categorias)                # muestra los bloques en los que se dividen los valores
print(categorias.value_counts()) # muestra la cantidad de valores en cada bloque

In [ ]:
bins = 5
edades = [1,6,8,16,25,18,71,44,100]
categorias = pd.qcut(edades,bins) # rangos homogéneos (similar número de valores). qcut divide los valores en bloques de igual tamaño.
print(categorias)
print(categorias.value_counts())

## <img src="img/by-nc.png" width="200">